# Creating and plotting elemental EELS maps

In this tutorial, elemental maps of copper and zinc will be created from electron energy loss spectra (EELS). The data is recorded using a sample of copper and zinc oxide deposited on carbon nanotubes. The particles are likely to be very small, and the carbon is not completely covered. In this sample, the ratio of Zn and Cu is 3:1, and approximately 80 wt% of the sample is carbon. Due to the small amount of copper and zinc, the signal strength is low. To improve the signal strength without lowering the spatial resolution, the spectra were recorded with a high energy dispersion, resulting in low energy resolution. This means that any fine structure is not visible.

For more information about the material system and how the results of this processing can be used, see the paper: https://doi.org/10.1016/j.cattod.2019.02.045

This notebook requires `HyperSpy` 2.0 or higher. It also requires `eXSpy` which is a library which contains all of HyperSpy's EELS and EDX functionality.

The main objective is to show how the functionalities of HyperSpy and eXSpy can be used to create relative elemental maps from EEL spectra.



### Changes

* 2017/09/27: Initial version by Ida Hjorth
* 2019/11/14: Update to HyperSpy 1.5, and minor improvements to text by Magnus Nord
* 2024/3/16: Update to work with HyperSpy 2.0, by Magnus Nord

### Table of contents

1. <a href='#load'> Load data</a> 
2. <a href='#fitting'>Fitting Cu and Zn Hartree-Slater cross sections</a>
3. <a href='#save_restore'>Saving and restoring the model</a>
4. <a href='#plotting'>Creating elemental maps from the model</a>

# <a id='load'></a>1. Load data

In [ ]:
%matplotlib widget
import numpy as np
import hyperspy.api as hs

In [ ]:
s_eels = hs.load('datasets/CuZn_EELS_mapping_tutorial.hspy')

Making elemental maps for this map is a bit tricky, due to the fairly weak edges, and the overlap between the Cu and Zn edges.

In [ ]:
s_eels.plot()

# Generate synthetic zero loss peak for low-loss signal (if lacking)

In the model-based approach, the low-loss signal can be convolved with the high-loss signal such that the energy spread of the electron beam and plural scattering due to the bulk plasmon are taken into account. This will lead to a better fit of the model to the experimental data. In addition, the experimental zero loss peak can be used to precisely calibrate the energy scale. If you have recorded a low-loss signal, you can skip this part and go directly to the generation of the background model.

Unfortunately, in our example the low-loss signal was not recorded. Since the core loss components we'll be fitting to data have sharp onsets and sharp features, we make a synthetic low loss signal to take into account the energy spread of the beam.

We give this Gaussian component a full width at half maximum (FWHM) equivalent to approximately the energy spread of the beam, acquired at a fairly high spectrometer dispersion (~4 eV)
and a low loss signal with the same spatial dimensions as the high-loss EELS signal. This is the second best option, after of course having a real low-loss signal.

In [ ]:
g = hs.model.components1D.Gaussian(A=1., centre=0.) # Zero loss peak
g.fwhm = 5

The zero-loss signal `s_zlg` can be generated as follows. This synthetic signal accounts for the energy spread of the electron beam, but not plural scattering as would with a recorded low-loss signal.

In [ ]:
energy_axis = np.arange(-50, 200, 1)
shape = (s_eels.axes_manager.navigation_axes[1].size, s_eels.axes_manager.navigation_axes[0].size, len(energy_axis))
lowloss = np.zeros(shape)
lowloss[:, :] = g.function(energy_axis)
s_zlg = hs.signals.Signal1D(lowloss)
s_zlg.set_signal_type("EELS")
s_zlg.axes_manager.signal_axes[0].offset = -50

In [ ]:
s_zlg.plot()

# <a id='fitting'></a>2. Fitting Cu and Zn components to the spectrum

A model consisting of the powerlaw background Cu and Zn cross sections will be created.

**Note**: this will download a set of open source **Generalised Oscillator Strengths (GOS)** , which was computed and packaged by Leonhard Segger, Giulio Guzzinati and Helmut Kohl. For more information about the GOS files, see the [Zenodo deposit](https://zenodo.org/doi/10.5281/zenodo.6599070). For more information about how these were computed, and the code used to compute them, see https://github.com/Br0Fi/goscalc.

Next, the experimental microscope parameters is set, and the model is created.

In [ ]:
s_eels.set_microscope_parameters(beam_energy=200, convergence_angle=13.33, collection_angle=55.28) 
s_eels.add_elements(('Cu', 'Zn')) 
m = s_eels.create_model(low_loss=s_zlg)

Both the power law background and core less edge components (`EELSCLEdge`) has automatically been added.

In [ ]:
m.components

Due to the large intensity variations across the dataset, the different features are fitted one by one with initial values and bounds for the parameters. After having run the fitting for the individual features, all of them will be fitted at the same time.

We start with the power law background, where the following bounding and initial values were found to work well on the dataset used in this tutorial.

In [ ]:
m.fit_component(m.components.PowerLaw)

In [ ]:
m.plot(plot_components=True)

Then the power law is frozen, so the Cu and Zn `EELSCLEdge` components can be fitted without the power law background changing.

In [ ]:
m.components.PowerLaw.set_parameters_not_free()

Due to the varying and low signal strength, some initial parameters and boundaries are set to ensure successful fitting. 

In [ ]:
m.fit_component(m.components.Cu_L3)

In [ ]:
m.fit_component(m.components.Zn_L3)

Lastly, the power law, the Cu and Zn components are fitted together over the whole energy range.

In [ ]:
m.multifit()

In [ ]:
m.components.PowerLaw.set_parameters_free(parameter_name_list=['A', 'r'])
m.components.Cu_L3.intensity.free = True
m.reset_signal_range()
m.multifit(bounded=True)

Let's see the results. Go to the region around `(32, 14)`, where there clearly is copper or zinc of some kind.

In [ ]:
m.plot(plot_components=True)

# 3. <a id='save_restore'></a>Saving and restoring the models

As the fitting process can be slow, saving the models can be a good idea. 

In [ ]:
m.save('model.hspy', overwrite=True)

This results in a file called `model.hspy`, which can be loaded and restored.

In [ ]:
mr = hs.load('model.hspy')
mr = mr.models.restore('a')

In [ ]:
mr.plot()

# 4. <a id='plotting'></a>Make maps

In this section, elemental maps will be plotted from the intensity of the model of the Cu-L$_3$ and Zn-L$_3$ components saved above. The HAADF image will also be plotted, for comparison with the elemental distribution.

**Note:** as the components L$_3$, L$_2$ and L$_1$ are connected during the fitting procedure, we only need to visualize one of them to get a relative elemental map.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

In [ ]:
m = hs.load('model.hspy')
mr = m.models.restore('a')

In [ ]:
s_copper = mr.components.Cu_L3.intensity.as_signal()
s_zinc = mr.components.Zn_L3.intensity.as_signal()
s_haadf = hs.load('datasets/CuZn_HAADF.hspy').as_signal2D((0, 1))

`GridSpec` is a convenient function to use when creating several subplots. We create axes for the elemental maps, HAADF signal as well as the colorbars that will be used indicate the numerical intensity.

For more information on how to use `GridSpec`, you can access the docstring by running `gridspec.GridSpec?` in a cell.  (Note the question mark)

In [ ]:
fig = plt.figure(figsize=(5.5, 2.7))
gs = gridspec.GridSpec(30, 3)
ax_cu = fig.add_subplot(gs[0:-2, 0])
ax_zn = fig.add_subplot(gs[0:-2, 1])
ax_haadf = fig.add_subplot(gs[0:-2, 2])
cbar_cu =  fig.add_subplot(gs[-1, 0]) # Colorbars are much thinner than the map axes (1/30 of the height of the image)
cbar_zn =  fig.add_subplot(gs[-1, 1])
cbar_haadf =  fig.add_subplot(gs[-1, 2])

The signals can be plotted by using `imshow`.

In [ ]:
cax_cu = ax_cu.imshow(
    s_copper.data,
    interpolation='nearest',
    origin='lower',
    extent=s_copper.axes_manager.signal_extent,
)

In [ ]:
cax_zn = ax_zn.imshow(
    s_zinc.data,
    interpolation='nearest',
    origin='lower',
    extent=s_zinc.axes_manager.signal_extent,
)

NumPy functions, such as `flipud`, `fliplr` and `rot90` can also be used to give the image the correct orientation. Here,`flipud` is used to give the HAADF image the same orientation as the elemental maps.

In [ ]:
ax_haadf.imshow(
    np.flipud(s_haadf.data),
    interpolation='nearest',
    origin='upper',
    extent=s_haadf.axes_manager.signal_extent,
)

Disable axis ticks

In [ ]:
ax_haadf.set_xticks([])
ax_cu.set_xticks([])
ax_zn.set_xticks([])
ax_zn.set_yticks([])
ax_cu.set_yticks([])
ax_haadf.set_yticks([])

Scalebars can be added by using `AnchoredSizeBar`. 

In [ ]:
fontprops = fm.FontProperties(size=18)
scalebar0 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 1, # length of bar, label, loc
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops,
)
scalebar1 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 1,
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops,
)
scalebar2 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 1,
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops,
)
ax_cu.add_artist(scalebar0)
ax_zn.add_artist(scalebar1)
ax_haadf.add_artist(scalebar2)

We add labels to indicate what is shown in each image.

In [ ]:
ax_cu.text(0.05, 0.05, 'Cu', color='white', size=18, transform=ax_cu.transAxes)
ax_zn.text(0.05, 0.05, 'Zn', color='white', size=18, transform=ax_zn.transAxes)
ax_haadf.text(0.05, 0.05, 'HAADF', color='white', size=18, transform=ax_haadf.transAxes)

Then colorbars are added. 

In [ ]:
cb_zn = fig.colorbar(ax_zn.images[0], cax=cbar_zn, extend='both', orientation='horizontal', label="Relative Zn, [a.u.]")
cb_cu = fig.colorbar(ax_cu.images[0], cax=cbar_cu,extend='both', orientation='horizontal', label="Relative Cu, [a.u.]")
cb_haadf = fig.colorbar(ax_haadf.images[0], cax=cbar_haadf, extend='both', orientation='horizontal', label="HAADF, [a.u.]")

In [ ]:
cb_cu.set_ticks([0, 5, 10, 15])
cb_zn.set_ticks([0, 5, 10, 15, 20, 25])
cb_haadf.set_ticks([0, 1000, 2000])

The color schemes `viridis`, `inferno`, `plasma` and `magma` are nice to use as they are grayscale compatible, perceptually uniform and colorblind-proof. For more info on these color maps, see https://www.youtube.com/watch?v=xAoljeRJ3lU

In [ ]:
cax_zn.set_clim(vmin=0, vmax=25)
cax_zn.set_cmap('inferno')
cax_cu.set_clim(vmin=0, vmax=15)
cax_cu.set_cmap('viridis')

In [ ]:
fig.subplots_adjust(left=0, bottom=0.15, right=1, top=0.98)

Saving the matplotlib figure object as a png-file. Here, the resolution (via dots per inch, `dpi`) is set to a high value: 300. 

In [ ]:
fig.savefig("elemental_map.png", dpi=300)

We can see the end result inside the Notebook by using `plt.show()`Showing the end result via the `plt.

In [ ]:
plt.show()